In [1]:
%matplotlib inline

## Iris introduction course
# 2. Loading and Saving

**Learning Outcome**: by the end of this section, you will be able to use Iris to load datasets from disk as Iris cubes and save Iris cubes back to disk.

**Duration:** 1 hour

**Overview:**<br>
2.1 [Iris Load Functions](#iris_load_functions)<br>
2.2 [Saving Cubes](#saving)<br>
2.3 [Out-of-core Processing](#out_of_core_processing)<br>
2.4 [Exercise](#exercise_2)<br>
2.5 [Summary of the Section](#summary)

In [2]:
import iris

----

## 2.1 Iris Load Functions<a id='iris_load_functions'></a>

There are three main load functions in Iris: ``load``, ``load_cube`` and ``load_cubes``.

1. **load** is a general purpose loading function. Typically this is where all data analysis will start, before more loading is refined with the more controlled loading from the other two functions.
2. **load_cube** returns a single cube from the given source(s) and constraint. There will be exactly one cube, or an exception will be raised.
3. **load_cubes** returns a cubelist of cubes from the given sources(s) and constraint(s). There will be exactly one cube per constraint, or an exception will be raised.


Note: ``load_cube`` is a special case of ``load``. 

Let's compare the result of calling `load` and `load_cube`. We start by selecting the `air_temp.pp` file from Iris' sample data.

In [3]:
fname = iris.sample_data_path('air_temp.pp')

If we give this filepath to `load`, we see that Iris returns a cubelist.

In [4]:
c1 = iris.load(fname)
print(type(c1))

<class 'iris.cube.CubeList'>


If instead we give this filepath to `iris.load_cube` we see that Iris a cube.

In [5]:
c2 = iris.load_cube(fname)
print(type(c2))

<class 'iris.cube.Cube'>


If we compare the first cube in the cubelist returned by calling `load` and the cube returned by calling `load_cube` we see that they are the equal.

In [6]:
c1[0] == c2

True

<div class="alert alert-block alert-info">
<b>Suggested Activity: </b>Try loading in the `uk_hires.pp` from Iris' sample data, first with iris.load then iris.load_cube. Why does iris.load_cube fail if you only supply the filepath?
</div>

----

## 2.2 Saving Cubes<a id='saving'></a>

The [iris.save](https://scitools.org.uk/iris/docs/latest/iris/iris.html#iris.save) function provides a convenient interface to save Cube and CubeList instances.

Below we load the `uk_hires.pp` file from Iris' provided sample data which returns a cubelist of the cubes that were produced from that file. We then save this cubelist out to netcdf.

In [7]:
fname = iris.sample_data_path('uk_hires.pp')
cubes = iris.load(fname)
iris.save(cubes, 'saved_cubes.nc')

We can check the ncdump to see what Iris saved:

In [8]:
!ncdump -h saved_cubes.nc | head -n 20
!rm saved_cubes.nc

netcdf saved_cubes {
dimensions:
	time = 3 ;
	model_level_number = 7 ;
	grid_latitude = 204 ;
	grid_longitude = 187 ;
	bnds = 2 ;
variables:
	float air_potential_temperature(time, model_level_number, grid_latitude, grid_longitude) ;
		air_potential_temperature:standard_name = "air_potential_temperature" ;
		air_potential_temperature:units = "K" ;
		air_potential_temperature:um_stash_source = "m01s00i004" ;
		air_potential_temperature:grid_mapping = "rotated_latitude_longitude" ;
		air_potential_temperature:coordinates = "forecast_period forecast_reference_time level_height sigma surface_altitude" ;
	int rotated_latitude_longitude ;
		rotated_latitude_longitude:grid_mapping_name = "rotated_latitude_longitude" ;
		rotated_latitude_longitude:longitude_of_prime_meridian = 0. ;
		rotated_latitude_longitude:earth_radius = 6371229. ;
		rotated_latitude_longitude:grid_north_pole_latitude = 37.5 ;
		rotated_latitude_longitude:grid_north_pole_longitude = 177.5 ;


Extra keywords can be passed to specific fileformat savers.

<div class="alert alert-block alert-info">
<b>Suggested Activity: </b>Take a look at the above link to the `iris.save` documentation to see what fileformats iris supports saving to.
</div>

-----

## 2.3 Out-of-core Processing<a id='out_of_core_processing'></a>

[Out-of-core processing](https://en.wikipedia.org/wiki/External_memory_algorithm) is a technical term that describes being able to process datasets that are too large to fit in memory at once. In Iris, this functionality is referred to as **lazy data**. It means that you can use Iris to load, process and save datasets that are too large to fit in memory without running out of memory. This is achieved by loading only the dataset's metadata and not the data array, unless this is specifically requested.

To determine whether your cube has lazy data you can use the `has_lazy_data` method

In [9]:
fname = iris.sample_data_path('air_temp.pp')
cube = iris.load_cube(fname)
print(cube.has_lazy_data())

True


Iris tries to maintain lazy data as much as possible. We refer to the operation of loading a cube's lazy data as 'realising' the cube's data. A cube's lazy data will only be loaded in a limited number of cases, including:

* when the user directly requests the cube's data using `cube.data`,
* when there is no lazy data processing algorithm available to perform the requested data processing, such as for peak finding, and
* where actual data values are necessary, such as for cube plotting.

In [10]:
cube.data
print(cube.has_lazy_data())

False


Above we have triggered the data to be loaded into memory by calling `cube.data`.

----

## 2.4 Exercise 2<a id='exercise_2'></a>

1\. Load the file in `iris.sample_data_path('atlantic_profiles.nc')`, as in Exercise 1, but this time use `iris.load_cube` to load in the `sea_water_potential_temperature` cube only.

2\. Check whether this cube has lazy data? Print the minimum, maximum, mean and standard deviation of the cube's data. Does the cube still have lazy data?

3\. Go to the Iris reference documentation for ``iris.save``. What keywords are accepted to ``iris.save`` when saving a PP file?

#### Exercise Solutions: 
Once you are happy with your answers, you can check the solutions by runnning the cell below:

In [1]:
# SAMPLE SOLUTION
# %load solutions/iris_exercise_2

-----

## 2.5 Section Summary : Loading and Saving<a id='summary'></a>

In this section we learnt:
* Iris has different loading functions for different purposes, for example `iris.load` should be used for exploratory data analysis
* Iris supports saving to three fileformats: netCDF, GRIB2 and PP
* Iris uses lazy data and out-of-core-processing to handle data that it too large to fit into memory